In [ ]:
# %pip install -q psycopg2-binary
%pip install -q python-dotenv
%pip install -q langchain_community
%pip install -q langchain-openai
%pip install -q langgraph

Note: you may need to restart the kernel to use updated packages.


In [104]:
# import psycopg2
import os
from dotenv import load_dotenv

In [105]:
load_dotenv()
DATABASE_URL=os.environ.get("DATABASE_URL")
OPENAI_API_KEY=os.environ.get("OPENAI_API_KEY")

In [106]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri(DATABASE_URL)
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM customers LIMIT 1;")

postgresql
['customers', 'deliverypersonnel', 'feedbackcomplaints', 'fooditems', 'orderitems', 'orders', 'payments', 'restaurants']


"[(1, 'Ali Ahmed', 'ali.ahmed@example.com', '03001234567', 'Gulberg, Lahore', 'Visa ****1234', 120, datetime.datetime(2024, 10, 20, 12, 4, 37, 63089))]"

In [107]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [108]:
from langchain_core.prompts import SystemMessagePromptTemplate

system_template = """You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Always limit your query to at most {top_k} results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

To start you should ALWAYS look at the tables in the database to see what you can query.
Do NOT skip this step.
Then you should query the schema of the most relevant tables."""

system_template = SystemMessagePromptTemplate.from_template(system_template)
system_message = system_template.format(
    dialect="PostgreSQL", top_k=5
)
# system_message.pretty_print()

In [109]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)
tools = toolkit.get_tools()
tools

[QuerySQLDataBaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x124f32cd0>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x124f32cd0>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x124f32cd0>),
 QuerySQLCheckerTool(description='Use this tool to double check if your 

In [111]:
sql_db_schema=tools[1]
sql_db_schema("orders", "deliverypersonnel")

AttributeError: 'str' object has no attribute 'parent_run_id'

In [116]:
from langchain_core.messages import AIMessage, SystemMessage, ToolMessage

response = {'model': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_UHus4uQnJcH11ZcsBqV6VURU', 'function': {'arguments': '{"table_names": "orders"}', 'name': 'sql_db_schema'}, 'type': 'function'}, {'id': 'call_n6iaoZQKA52BdfZvbcaa5lnc', 'function': {'arguments': '{"table_names": "deliverypersonnel"}', 'name': 'sql_db_schema'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 595, 'total_tokens': 645, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0705bf87c0', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-1d386c11-fde4-4842-b4c8-d6a1e7dd2c54-0', tool_calls=[{'name': 'sql_db_schema', 'args': {'table_names': 'orders'}, 'id': 'call_UHus4uQnJcH11ZcsBqV6VURU', 'type': 'tool_call'}, {'name': 'sql_db_schema', 'args': {'table_names': 'deliverypersonnel'}, 'id': 'call_n6iaoZQKA52BdfZvbcaa5lnc', 'type': 'tool_call'}], usage_metadata={'input_tokens': 595, 'output_tokens': 50, 'total_tokens': 645, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}}, {'tools': {'messages': [ToolMessage(content='\nCREATE TABLE orders (\n\torderid SERIAL NOT NULL, \n\tcustomerid INTEGER, \n\trestaurantid INTEGER, \n\torderstatus VARCHAR(50), \n\ttotalcost DOUBLE PRECISION, \n\tdeliveryaddress VARCHAR(255), \n\torderdate TIMESTAMP WITHOUT TIME ZONE DEFAULT CURRENT_TIMESTAMP, \n\tpaymentmethod VARCHAR(100), \n\tspecialinstructions TEXT, \n\tCONSTRAINT orders_pkey PRIMARY KEY (orderid), \n\tCONSTRAINT orders_customerid_fkey FOREIGN KEY(customerid) REFERENCES customers (customerid), \n\tCONSTRAINT orders_restaurantid_fkey FOREIGN KEY(restaurantid) REFERENCES restaurants (restaurantid)\n)\n\n/*\n3 rows from orders table:\norderid\tcustomerid\trestaurantid\torderstatus\ttotalcost\tdeliveryaddress\torderdate\tpaymentmethod\tspecialinstructions\n1\t1\t1\tCompleted\t1200.5\tGulberg, Lahore\t2024-10-20 12:06:28.371072\tVisa\tNo onions please\n2\t2\t2\tIn Progress\t1500.75\tDHA, Karachi\t2024-10-20 12:06:28.371072\tEasypaisa\tExtra spicy\n3\t3\t3\tRejected\t800.25\tF-7, Islamabad\t2024-10-20 12:06:28.371072\tCOD\tNo special instructions\n*/', name='sql_db_schema', id='92d9981e-56ed-4bed-a0d6-be86564b2f58', tool_call_id='call_UHus4uQnJcH11ZcsBqV6VURU'), ToolMessage(content='\nCREATE TABLE deliverypersonnel (\n\tdeliveryid SERIAL NOT NULL, \n\tassignedorderids INTEGER[], \n\tname VARCHAR(255), \n\tcontactinformation VARCHAR(255), \n\tcurrentlocation VARCHAR(255), \n\tCONSTRAINT deliverypersonnel_pkey PRIMARY KEY (deliveryid)\n)\n\n/*\n3 rows from deliverypersonnel table:\ndeliveryid\tassignedorderids\tname\tcontactinformation\tcurrentlocation\n1\t[1, 4, 5]\tAhmed Ali\t0300-1234567\tGulberg, Lahore\n2\t[2, 8, 6]\tImran Khan\t0320-1234567\tDHA, Karachi\n3\t[3, 9, 7]\tAsad Khan\t0301-1234567\tBlue Area, Islamabad\n*/', name='sql_db_schema', id='11df687f-633e-463b-975a-a4d60d66b5a7', tool_call_id='call_n6iaoZQKA52BdfZvbcaa5lnc')]}}

In [141]:
from pprint import pprint
event1 = response[0]
event2 = response[1]

len(next(iter(event1.values()))['messages'])
next(iter(event2.values()))['messages'][1]
next(iter(event1.values()))["messages"]

ls=[(message) for event in response for message in next(iter(event.values()))['messages']]
ls

[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_UHus4uQnJcH11ZcsBqV6VURU', 'function': {'arguments': '{"table_names": "orders"}', 'name': 'sql_db_schema'}, 'type': 'function'}, {'id': 'call_n6iaoZQKA52BdfZvbcaa5lnc', 'function': {'arguments': '{"table_names": "deliverypersonnel"}', 'name': 'sql_db_schema'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 595, 'total_tokens': 645, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0705bf87c0', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-1d386c11-fde4-4842-b4c8-d6a1e7dd2c54-0', tool_calls=[{'name': 'sql_db_schema', 'args': {'table_names': 'orders'}, 'id': 'call_UHus4uQnJcH11ZcsBqV6VURU', 'type': 'tool_call'}, {'name':

In [126]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, START
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

memory = MemorySaver()

class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

llm_with_tools = llm.bind_tools(tools)

def assistant(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

graph_builder.add_node("assistant", assistant)

tool_node = ToolNode(tools=tools)
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "assistant",
    tools_condition,
)
graph_builder.add_edge("tools", "assistant")
graph_builder.add_edge(START, "assistant")
graph = graph_builder.compile(checkpointer=memory)

In [127]:
config = {"configurable": {"thread_id": "1"}}
graph.update_state(config=config, values={"messages": system_message})

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1ef97aac-2341-698a-8000-0e8045cc58bf'}}

In [ ]:
from langchain_core.messages import HumanMessage

def stream_graph_updates(user_input: str, config):
    # for event in graph.stream({"messages": [("user", user_input)]}, config, stream_mode="values"):
    #     for value in event.values():
    #         print("Assistant:", value["messages"][-1].content)
    events = graph.stream(
        {"messages": [HumanMessage(content=user_input)]}, config, stream_mode="values"
    )
    for event in events:
        if "messages" in event:
            event["messages"][-1].pretty_print()

while True:
    try:
        user_input = input("User: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break
        stream_graph_updates(user_input, config=config)
    except:
        print("Invalid Input!")
        break




================================ Human Message =================================

what is the status of order 3?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_qePnHUhVMGXL5QZVSdmP2J2g)
 Call ID: call_qePnHUhVMGXL5QZVSdmP2J2g
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

customers, deliverypersonnel, feedbackcomplaints, fooditems, orderitems, orders, payments, restaurants
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_c9FSeyblQNCLY4A76xQJRIjp)
 Call ID: call_c9FSeyblQNCLY4A76xQJRIjp
  Args:
    table_names: orders
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE orders (
	orderid SERIAL NOT NULL, 
	customerid INTEGER, 
	restaurantid INTEGER, 
	orderstatus VARCHAR(50), 
	totalcost DOUBLE PRECISION, 
	deliveryaddres

In [ ]:
from langchain_core.messages import AIMessage
snapshot = graph.get_state(config=config)
for message in snapshot.values["messages"]:
    if message.type in ['ai', 'human'] and 'tool_calls' not in message.additional_kwargs:
        message.pretty_print()

================================ Human Message =================================

what is the status of order number 3?
================================== Ai Message ==================================

The status of order number 3 is **Rejected**.
================================ Human Message =================================

what is the status of order number 3?
================================== Ai Message ==================================

The status of order number 3 is **Rejected**.
================================ Human Message =================================

what is the status of order number 3?
================================ Human Message =================================

What is the status of order 3?
================================ Human Message =================================

what is the status of order 3?
================================== Ai Message ==================================

The status of order number 3 is **Rejected**.
==============================

In [136]:
# Conversation 1: Order Status and Details
order_status_details = [
    "Hi, my name is Umar Siddiqui and my CustomerID is 7. Can you check the status of my latest order?",
    "How much did I spend on my last order?",
    "What items did I order last time?",
    "Who is delivering my order, and how can I contact them?",
    "Are there any pending payments on my order?",
    "What’s the expected delivery time for my current order?",
    "Can I add special instructions to my ongoing order?",
    "Is there a way to add or change items in my order now?"
]

config = {"configurable": {"thread_id": "2"}}
graph.update_state(config=config, values={"messages": system_message})
for user_input in order_status_details:
    stream_graph_updates(user_input, config=config)

================================ Human Message =================================

Hi, my name is Umar Siddiqui and my CustomerID is 7. Can you check the status of my latest order?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_oiFwcCBlpvIN1BXnOgHpTHtr)
 Call ID: call_oiFwcCBlpvIN1BXnOgHpTHtr
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

customers, deliverypersonnel, feedbackcomplaints, fooditems, orderitems, orders, payments, restaurants
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_kgMkjuk5N63U452HJOS1tRX0)
 Call ID: call_kgMkjuk5N63U452HJOS1tRX0
  Args:
    table_names: customers, orders
  sql_db_schema (call_vnQw5rcOeod76J1pO9FJPVRy)
 Call ID: call_vnQw5rcOeod76J1pO9FJPVRy
  Args:
    table_names: orderitems
================================= Tool Message =====================

In [ ]:

# Conversation 2: Account and Loyalty Points
account_loyalty_points = [
    "How many loyalty points do I have?",
    "Can you tell me when I last earned loyalty points?",
    "What’s my current address on file?",
    "How can I update my payment information?",
    "Can I redeem loyalty points on my next order?",
    "What’s my account’s email address?",
    "How do I reset my password?",
    "How often can I earn loyalty points?"
]

# Conversation 3: Payment and Billing Queries
payment_billing_queries = [
    "What payment method did I use for my last order?",
    "Are there any pending payments I need to clear?",
    "Can I change my payment method for an ongoing order?",
    "How do I view all my past transactions?",
    "Did I pay using Visa or JazzCash last time?",
    "Can I add a new credit card to my account?",
    "Why did my last payment fail?",
    "How can I get a receipt for my last purchase?"
]

# Conversation 4: Restaurant Information and Availability
restaurant_info_availability = [
    "What are the operating hours of Lahore Tikka House?",
    "Is there a restaurant that serves Afghani cuisine nearby?",
    "Which restaurants have the best rating?",
    "Are there any dessert options at Quetta Sajji House?",
    "Can you list all the restaurants available in Karachi?",
    "What type of food does Islamabad Café serve?",
    "Is there a place with high ratings that’s open late tonight?",
    "How far is Hyderabad Biryani Corner from my location?"
]

# Conversation 5: Food and Menu Queries
food_menu_queries = [
    "What’s on the menu at Karachi Broast?",
    "How much does a Beef Biryani cost at Hyderabad Biryani Corner?",
    "Can you suggest something spicy from Lahore Tikka House?",
    "Do any restaurants offer vegetarian options?",
    "What are the most popular items at Peshawar Chapli Kebab?",
    "Can I get a dessert recommendation from Islamabad Café?",
    "How many items does the Quetta Sajji House menu have?",
    "Are there any appetizers in the Pakistani cuisine options?"
]

# Conversation 6: Delivery Information
delivery_information = [
    "Who’s delivering my order right now?",
    "When should I expect my delivery?",
    "Can you provide contact details for the delivery person?",
    "Where is my delivery currently?",
    "What address is my order being delivered to?",
    "Is there a way to track the delivery in real-time?",
    "Has the delivery personnel started the delivery yet?",
    "Can I change the delivery address after placing the order?"
]